In [7]:
import pandas as pd
import numpy as np
import scipy as sp

In [90]:
# import sys
# !{sys.executable} -m pip install BayesPy

In [91]:
import seaborn as sns
from matplotlib import pyplot as plt

In [475]:
import numpy as np

class UGMM(object):
    
    def __init__(self, X, K=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]

    def _init(self):
        self.alpha=np.random.rand(self.K,1)
        self.m= np.random.rand(self.K,1)
        self.beta=np.random.rand(self.K,1)
        self.beta_prev=np.random.rand(self.K,1)
        self.nu=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.W=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.alpha=np.random.rand(self.K,1)
        self.r=np.zeros((self.N,self.K))
        self.x_nebla=np.zeros((self.K,1))
        self.Nk=np.zeros((self.K,1))
        self.S=np.zeros((self.K,1))
        self.pi_nebla=np.zeros((self.K,1))
        self.lamda_nebla=np.zeros((self.K,1))
        self.E_mu_lamda=np.zeros((self.N,self.K))
        
        

        print('Init mean')
        print(self.m)
        print('Init lamda')


    def E_Step(self):
        for n in range(self.N):
            for k in range(self.K):
                self.pi_nebla[k]=np.exp(sp.special.digamma(self.alpha[k])-sp.special.digamma(self.alpha.sum()))
                self.lamda_nebla[k]=np.exp(sp.special.digamma(self.nu[k]/2, out=None)+np.log(2)+np.log(self.W[k]))
                self.E_mu_lamda[n][k]=(self.beta[k]**(-1))+self.nu[k]*self.W[k]*(self.X[n]-self.m[k])**2
                self.r[n][k]=self.pi_nebla[k]*(self.lamda_nebla[k]**0.5)*(np.exp(-0.5*self.E_mu_lamda[n][k]))
        row_sums = self.r.sum(axis=1)
        self.r= self.r / row_sums[:, np.newaxis]
        
        
    def fit(self, max_iter=1000, tol=1e-6):
        self._init()
#         self.r_values = [self.r_elbo()]
        self.m_history = [self.m.copy()]
        self.E_Step()
#         self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
#             print('before')
#             print(self.m_history)
            self.M_Step()
#             print('after')
#             print(self.m_history)
            self.m_history.append(self.m.copy())
#             if iter_ % 2 == 0:
#             print('wow')
            print(iter_, self.m_history[iter_])
            print(iter_, self.r)
            if np.linalg.norm(self.m_history[-1]-self.m_history[-2],2) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.m_history[-1][0],iter_))
                break
            self.E_Step()
        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.m_history[-1][0]))


    def M_Step(self):
        self._update_Nk()
        self._update_x_nebla()
        self._update_S()
        self._update_alpha()
        self._update_W()
        self._update_beta()
        self._update_nu()
        self._update_m()

    def _update_Nk(self):
        for k in range(self.K):
            self.Nk[k]=self.r[:,k].sum()
            
    def _update_x_nebla(self):
        for k in range(self.K):
            self.x_nebla[k]=0
            for n in range(self.N):
#                 print(self.Nk)
                self.x_nebla[k]+=self.r[n][k]*self.X[n]/self.Nk[k]
    
    def _update_S(self):
        for k in range(self.K):
            self.S[k]=0
            for n in range(self.N):
                self.S[k]+=(self.r[n][k]*(self.X[n]-self.x_nebla[k])**(2))/self.Nk[k]
                
    def _update_alpha(self):
        for k in range(self.K):
            self.alpha[k]=self.alpha[k]+self.Nk[k]
            
    def _update_beta(self):
        self.beta_prev=self.beta.copy()
        for k in range(self.K):
            self.beta[k]=self.beta[k]+self.Nk[k]
        
    def _update_m(self):
        for k in range(self.K):
            self.m[k]=(self.beta_prev[k]*self.m[k]+self.Nk[k]*self.x_nebla[k])/self.beta[k]
#         print('achha')
#         print(self.m)
    
    
    def _update_W(self):
        for k in range(self.K):
            self.W[k]=((1/self.W[k])+self.Nk[k]*self.S[k]+(self.beta[k]*self.Nk[k]/(self.beta[k]+self.Nk[k]))*(self.x_nebla[k]-self.m[k])**2)**(-1)
        
    
    def _update_nu(self):
        for k in range(self.K):
            self.nu[k]=self.nu[k]+self.Nk[k]
    
#     def _calculate_rnk
        

In [476]:
data=pd.read_csv('data2.txt',header=None)

In [477]:
X =np.array(data.iloc[0,0])
for i in range(1,1000):
    X= np.append(X, data.iloc[i,0])
    

In [478]:
X.max()

0.9942949885119834

In [480]:
ugmm = UGMM(X, 3)
ugmm.fit()

Init mean
[[0.83346122]
 [0.54705876]
 [0.63144309]]
Init lamda
1 [[0.62031465]
 [0.45238812]
 [0.44555295]]
1 [[1.17185078e-04 9.79736548e-01 2.01462671e-02]
 [8.67458680e-05 9.78643912e-01 2.12693425e-02]
 [7.98104420e-05 9.78231894e-01 2.16882951e-02]
 ...
 [1.63598168e-04 9.79962219e-01 1.98741829e-02]
 [2.07435945e-04 9.78793656e-01 2.09989079e-02]
 [1.31386072e-04 9.79959680e-01 1.99089343e-02]]
2 [[0.61974716]
 [0.45210543]
 [0.46021921]]
2 [[7.43771069e-07 9.96013956e-01 3.98530069e-03]
 [7.90433942e-07 9.94933004e-01 5.06620566e-03]
 [8.46093891e-07 9.94261752e-01 5.73740162e-03]
 ...
 [1.18312820e-06 9.94425387e-01 5.57343032e-03]
 [3.42433332e-06 9.83776696e-01 1.62198797e-02]
 [8.03457276e-07 9.95924207e-01 4.07498957e-03]]
3 [[0.61947484]
 [0.45202521]
 [0.46824928]]
3 [[3.79760446e-07 9.96871428e-01 3.12819227e-03]
 [4.05361955e-07 9.96028906e-01 3.97068892e-03]
 [4.34987641e-07 9.95501851e-01 4.49771449e-03]
 ...
 [6.08990161e-07 9.95589743e-01 4.40964796e-03]
 [1.806810

26 [[0.61826892]
 [0.45196868]
 [0.49928426]]
26 [[3.34632483e-08 9.98088580e-01 1.91138635e-03]
 [3.58759646e-08 9.97689742e-01 2.31022210e-03]
 [3.85813622e-08 9.97430267e-01 2.56969444e-03]
 ...
 [5.39026718e-08 9.97289652e-01 2.71029397e-03]
 [1.62873451e-07 9.92463163e-01 7.53667449e-03]
 [3.61796884e-08 9.98023485e-01 1.97647928e-03]]
27 [[0.61824924]
 [0.45196946]
 [0.49961182]]
27 [[3.22292775e-08 9.98092175e-01 1.90779250e-03]
 [3.45537875e-08 9.97697422e-01 2.30254386e-03]
 [3.71597218e-08 9.97440230e-01 2.55973281e-03]
 ...
 [5.19136171e-08 9.97294745e-01 2.70520302e-03]
 [1.56862491e-07 9.92493923e-01 7.50592056e-03]
 [3.48452441e-08 9.98026713e-01 1.97325219e-03]]
28 [[0.61823028]
 [0.45197023]
 [0.49992006]]
28 [[3.10849528e-08 9.98095339e-01 1.90463015e-03]
 [3.33276243e-08 9.97704492e-01 2.29547461e-03]
 [3.58413017e-08 9.97449476e-01 2.55048808e-03]
 ...
 [5.00690744e-08 9.97299231e-01 2.70071850e-03]
 [1.51287907e-07 9.92522583e-01 7.47726607e-03]
 [3.36077534e-08 9.9

50 [[0.6179298 ]
 [0.45198295]
 [0.50387657]]
50 [[1.75699908e-08 9.98110931e-01 1.88905153e-03]
 [1.88430299e-08 9.97780258e-01 2.21972323e-03]
 [2.02656950e-08 9.97557603e-01 2.44237708e-03]
 ...
 [2.82865121e-08 9.97322219e-01 2.67775244e-03]
 [8.54395440e-08 9.92873393e-01 7.12652120e-03]
 [1.89932803e-08 9.98037268e-01 1.96271266e-03]]
51 [[0.61791958]
 [0.45198339]
 [0.50398093]]
51 [[1.72336497e-08 9.98110457e-01 1.88952607e-03]
 [1.84824801e-08 9.97781816e-01 2.21816571e-03]
 [1.98779605e-08 9.97560215e-01 2.43976508e-03]
 ...
 [2.77445173e-08 9.97321613e-01 2.67835902e-03]
 [8.38009324e-08 9.92882557e-01 7.11735925e-03]
 [1.86296020e-08 9.98036465e-01 1.96351671e-03]]
52 [[0.61790956]
 [0.45198382]
 [0.50408137]]
52 [[1.69102305e-08 9.98109936e-01 1.89004710e-03]
 [1.81357783e-08 9.97783289e-01 2.21669317e-03]
 [1.95051169e-08 9.97562720e-01 2.43726041e-03]
 ...
 [2.72233526e-08 9.97320943e-01 2.67902990e-03]
 [8.22253018e-08 9.92891402e-01 7.10851535e-03]
 [1.82798975e-08 9.9

74 [[0.61772783]
 [0.45199144]
 [0.50559318]]
74 [[1.20003344e-08 9.98091760e-01 1.90822822e-03]
 [1.28719260e-08 9.97801686e-01 2.19830144e-03]
 [1.38442142e-08 9.97599652e-01 2.40033447e-03]
 ...
 [1.93124991e-08 9.97296975e-01 2.70300559e-03]
 [5.83101597e-08 9.93031789e-01 6.96815257e-03]
 [1.29711755e-08 9.98010891e-01 1.98909605e-03]]
75 [[0.61772092]
 [0.45199172]
 [0.50563945]]
75 [[1.18452498e-08 9.98090774e-01 1.90921447e-03]
 [1.27056451e-08 9.97802110e-01 2.19787763e-03]
 [1.36653863e-08 9.97600776e-01 2.39921053e-03]
 ...
 [1.90626654e-08 9.97295670e-01 2.70431130e-03]
 [5.75549765e-08 9.93036501e-01 6.96344147e-03]
 [1.28035013e-08 9.98009634e-01 1.99035301e-03]]
76 [[0.61771411]
 [0.451992  ]
 [0.50568435]]
76 [[1.16942148e-08 9.98089782e-01 1.91020666e-03]
 [1.25437051e-08 9.97802511e-01 2.19747679e-03]
 [1.34912265e-08 9.97601867e-01 2.39811902e-03]
 ...
 [1.88193580e-08 9.97294356e-01 2.70562472e-03]
 [5.68195281e-08 9.93041114e-01 6.95882929e-03]
 [1.26402058e-08 9.9

99 [[0.6175782 ]
 [0.45199742]
 [0.50642661]]
99 [[9.05613205e-09 9.98066353e-01 1.93363767e-03]
 [9.71497522e-09 9.97807259e-01 2.19273134e-03]
 [1.04490026e-08 9.97620384e-01 2.37960542e-03]
 ...
 [1.45700883e-08 9.97263435e-01 2.73655017e-03]
 [4.39766305e-08 9.93126814e-01 6.87314243e-03]
 [9.78806773e-09 9.97979305e-01 2.02068496e-03]]
100 [[0.61757304]
 [0.45199762]
 [0.50644929]]
100 [[8.96870238e-09 9.98065338e-01 1.93465299e-03]
 [9.62122132e-09 9.97807329e-01 2.19266093e-03]
 [1.03481714e-08 9.97620978e-01 2.37901179e-03]
 ...
 [1.44292801e-08 9.97262101e-01 2.73788501e-03]
 [4.35511111e-08 9.93129874e-01 6.87008273e-03]
 [9.69354729e-09 9.97978066e-01 2.02192444e-03]]
101 [[0.61756792]
 [0.45199782]
 [0.50647136]]
101 [[8.88298433e-09 9.98064325e-01 1.93566639e-03]
 [9.52930245e-09 9.97807392e-01 2.19259832e-03]
 [1.02493136e-08 9.97621559e-01 2.37843116e-03]
 ...
 [1.42912299e-08 9.97260769e-01 2.73921684e-03]
 [4.31339303e-08 9.93132892e-01 6.86706495e-03]
 [9.60087752e-09

...

900 [[0.61643752]
 [0.45203188]
 [0.50544165]]
900 [[1.09370347e-09 9.97751866e-01 2.24813282e-03]
 [1.17420697e-09 9.97743609e-01 2.25638966e-03]
 [1.26307600e-09 9.97667856e-01 2.33214266e-03]
 ...
 [1.75582146e-09 9.96880569e-01 3.11942894e-03]
 [5.28619757e-09 9.93767038e-01 6.23295665e-03]
 [1.18145710e-09 9.97610007e-01 2.38999151e-03]]
901 [[0.61643693]
 [0.45203189]
 [0.50543961]]
901 [[1.09254650e-09 9.97751700e-01 2.24829937e-03]
 [1.17296533e-09 9.97743565e-01 2.25643368e-03]
 [1.26174047e-09 9.97667863e-01 2.33213598e-03]
 ...
 [1.75396231e-09 9.96880384e-01 3.11961400e-03]
 [5.28059451e-09 9.93767383e-01 6.23261170e-03]
 [1.18020700e-09 9.97609815e-01 2.39018422e-03]]
902 [[0.61643635]
 [0.4520319 ]
 [0.50543757]]
902 [[1.09139204e-09 9.97751533e-01 2.24846572e-03]
 [1.17172638e-09 9.97743521e-01 2.25647765e-03]
 [1.26040783e-09 9.97667869e-01 2.33212931e-03]
 ...
 [1.75210720e-09 9.96880199e-01 3.11979881e-03]
 [5.27500361e-09 9.93767728e-01 6.23226712e-03]
 [1.17895960e-

924 [[0.61642372]
 [0.45203218]
 [0.50539323]]
924 [[1.06661242e-09 9.97747923e-01 2.25207547e-03]
 [1.14513309e-09 9.97742567e-01 2.25743218e-03]
 [1.23180365e-09 9.97668013e-01 2.33198579e-03]
 ...
 [1.71228895e-09 9.96876194e-01 3.12380452e-03]
 [5.15500226e-09 9.93775215e-01 6.22477957e-03]
 [1.15218524e-09 9.97605446e-01 2.39455293e-03]]
925 [[0.61642316]
 [0.45203219]
 [0.50539123]]
925 [[1.06551347e-09 9.97747762e-01 2.25223732e-03]
 [1.14395370e-09 9.97742524e-01 2.25747499e-03]
 [1.23053508e-09 9.97668019e-01 2.33197940e-03]
 ...
 [1.71052307e-09 9.96876014e-01 3.12398391e-03]
 [5.14968045e-09 9.93775551e-01 6.22444340e-03]
 [1.15099783e-09 9.97605259e-01 2.39474016e-03]]
926 [[0.61642259]
 [0.4520322 ]
 [0.50538924]]
926 [[1.06441684e-09 9.97747600e-01 2.25239898e-03]
 [1.14277680e-09 9.97742481e-01 2.25751775e-03]
 [1.22926918e-09 9.97668026e-01 2.33197303e-03]
 ...
 [1.70876092e-09 9.96875835e-01 3.12416307e-03]
 [5.14436990e-09 9.93775887e-01 6.22410758e-03]
 [1.14981292e-

948 [[0.61641028]
 [0.45203247]
 [0.50534585]]
948 [[1.04086349e-09 9.97744091e-01 2.25590809e-03]
 [1.11749931e-09 9.97741553e-01 2.25844626e-03]
 [1.20208027e-09 9.97668163e-01 2.33183569e-03]
 ...
 [1.67091414e-09 9.96871951e-01 3.12804766e-03]
 [5.03031345e-09 9.93783187e-01 6.21680832e-03]
 [1.12436371e-09 9.97601013e-01 2.39898609e-03]]
949 [[0.61640972]
 [0.45203248]
 [0.50534389]]
949 [[1.03981826e-09 9.97743933e-01 2.25606548e-03]
 [1.11637757e-09 9.97741511e-01 2.25848792e-03]
 [1.20087370e-09 9.97668169e-01 2.33182957e-03]
 ...
 [1.66923463e-09 9.96871777e-01 3.12822168e-03]
 [5.02525209e-09 9.93783514e-01 6.21648051e-03]
 [1.12323435e-09 9.97600831e-01 2.39916812e-03]]
950 [[0.61640917]
 [0.45203249]
 [0.50534194]]
950 [[1.03877519e-09 9.97743776e-01 2.25622268e-03]
 [1.11525813e-09 9.97741469e-01 2.25852953e-03]
 [1.19966961e-09 9.97668175e-01 2.33182347e-03]
 ...
 [1.66755858e-09 9.96871603e-01 3.12839549e-03]
 [5.02020115e-09 9.93783842e-01 6.21615304e-03]
 [1.12210732e-

972 [[0.61639716]
 [0.45203275]
 [0.50529947]]
972 [[1.01635862e-09 9.97740363e-01 2.25963617e-03]
 [1.09120042e-09 9.97740566e-01 2.25943324e-03]
 [1.17379268e-09 9.97668307e-01 2.33169187e-03]
 ...
 [1.63153933e-09 9.96867833e-01 3.13216529e-03]
 [4.91165537e-09 9.93790962e-01 6.20903340e-03]
 [1.09788653e-09 9.97596701e-01 2.40329742e-03]]
973 [[0.61639662]
 [0.45203276]
 [0.50529756]]
973 [[1.01536324e-09 9.97740210e-01 2.25978931e-03]
 [1.09013217e-09 9.97740525e-01 2.25947380e-03]
 [1.17264364e-09 9.97668313e-01 2.33168601e-03]
 ...
 [1.62993996e-09 9.96867664e-01 3.13233423e-03]
 [4.90683566e-09 9.93791282e-01 6.20871357e-03]
 [1.09681104e-09 9.97596524e-01 2.40347451e-03]]
974 [[0.61639608]
 [0.45203278]
 [0.50529565]]
974 [[1.01436986e-09 9.97740057e-01 2.25994229e-03]
 [1.08906605e-09 9.97740485e-01 2.25951431e-03]
 [1.17149691e-09 9.97668319e-01 2.33168015e-03]
 ...
 [1.62834380e-09 9.96867495e-01 3.13250297e-03]
 [4.90202563e-09 9.93791601e-01 6.20839407e-03]
 [1.09573771e-

996 [[0.61638436]
 [0.45203303]
 [0.50525407]]
996 [[9.93009091e-10 9.97736734e-01 2.26326480e-03]
 [1.06614125e-09 9.97739605e-01 2.26039438e-03]
 [1.14683852e-09 9.97668445e-01 2.33155389e-03]
 ...
 [1.59402187e-09 9.96863835e-01 3.13616384e-03]
 [4.79859772e-09 9.93798549e-01 6.20144595e-03]
 [1.07265785e-09 9.97592506e-01 2.40749290e-03]]
997 [[0.61638383]
 [0.45203304]
 [0.5052522 ]]
997 [[9.92060052e-10 9.97736585e-01 2.26341390e-03]
 [1.06512272e-09 9.97739565e-01 2.26043389e-03]
 [1.14574296e-09 9.97668451e-01 2.33154826e-03]
 ...
 [1.59249699e-09 9.96863670e-01 3.13632796e-03]
 [4.79400263e-09 9.93798861e-01 6.20113375e-03]
 [1.07163244e-09 9.97592334e-01 2.40766528e-03]]
998 [[0.6163833 ]
 [0.45203305]
 [0.50525033]]
998 [[9.91112872e-10 9.97736436e-01 2.26356284e-03]
 [1.06410618e-09 9.97739526e-01 2.26047335e-03]
 [1.14464955e-09 9.97668456e-01 2.33154264e-03]
 ...
 [1.59097511e-09 9.96863507e-01 3.13649187e-03]
 [4.78941655e-09 9.93799173e-01 6.20082186e-03]
 [1.07060903e-

In [481]:
mean=ugmm.m

In [482]:
mean

array([[0.61638225],
       [0.45203307],
       [0.50524659]])